MappMarkV
Embry-Riddle Aeronautical University
A21 Project

In [5]:
from ipyleaflet import *
from ipywidgets import *
from random import *

import pandas as pd
import numpy as np
import math

import xarray as xr
import os
import scipy

import pyqtree as qt
import bbox as bb

from collections import *
import operator
import time
from datetime import timedelta,datetime


In [1013]:
class mappDataFrame():
    
    
#the constructor takes in three parameters, which are defaulted to none; 
#dataType is an array which specifies the data type of a string, three options available, TFMS, ADSB, or AERO
#dataDate specifies the date for the mappDataFrame to pull from
#airport specifies the airport for the mappDataFrame to pull from
    
    def __init__(self, dataType = [], dataDate = None, airport = None):
    
        #the array below 
        self.dataType = []
        self.dataDate = None
        self.airport = None
        self.adsbDate = None
        self.adsbDateHyphen = None
        self.tfms = None
        self.adsbDf = None
        self.tfmsDf = None
        self.aeroDf = None
        self.uasFlights = None
        self.adiHigh = None
        self.adiLow = None
        self.timeOut = []
        self.adsbFlightsDictionary = {}
        self.adsbFlights = None
        self.adsbLines = None
        self.adsbDisplay = None
        self.adsbHeat = {}
        self.uasOd = None
        self.uasLines = None
        self.uasDisplay = None
        self.uasHeat = {}
        self.heatDistanceDelimiter = None
        self.altitudeDelimiter = None
        self.timeDelimiter = None
        self.encounterResults = None
        self.encounterFlights = None
        self.heatCircles = None
        self.heatDisplay = None
        self.mapp = None
        
        
#the below method is to be called after construction in order to pull the specified files and put them into workable dataframes
    
    def buildDataFrames(self, dataType, dataDate, airport = None):
        
        for dataTypes in dataType:
            self.dataType.append(dataTypes)
        
        
        #based on naming convention for adsb files
        if 'ADSB' in self.dataType:
            adsbDate = dataDate
            self.adsbDateHyphen = '-'.join(adsbDate)
            self.adsbDate = ''.join(adsbDate)
            self.adsbDf = pd.read_csv('./ADSB/' + self.adsbDate +'_csv.txt')
            
        
        #based on naming convention of tfms files
        if 'TFMS' in self.dataType:
            
            self.airport = airport
            self.tfmsDf = pd.read_csv('./TFMS/'+ self.airport + ".csv")
            
        #based on naming convention of aeroscope files
        if 'AERO' in self.dataType:
            
            xls = pd.ExcelFile('./AERO/aero8-14.xlsx')
            sheet_to_df_map = {}
            for sheet_name in xls.sheet_names:
                sheet_to_df_map[sheet_name] = xls.parse(sheet_name)
                sheet_to_df_map[sheet_name] = sheet_to_df_map[sheet_name].sort_values(['TimeWhen'])
                
            uasFlights = {}
            for df in sheet_to_df_map:
                
    
                
                testDf = pd.to_datetime(sheet_to_df_map[df]['TimeWhen'],#.astype('datetime64[ns]'),
                                        format = '%Y-%m-%d %H:%M:%S') 
                
                #The below offset is for the correction of time on the east coast. 
                #Aeroscope time data is provided in GMT time zone format
                
                testDf = testDf + pd.DateOffset(hours=6)
                
                
                
                
                #the line below stores the data in a dictionary that is organized by the drone ID
                uasFlights[str(sheet_to_df_map[df]['Drone ID'][0])] =[ [sheet_to_df_map[df]['Latitude'],
                                                                       sheet_to_df_map[df]['Longitude']],
                                                                      testDf.astype(str),
                                                                      sheet_to_df_map[df]['IconAltitude']]
                
            self.uasFlights = uasFlights

            
    
#the rebuildAdsbDf method is used to rebuild the adsbDataframe if a new date is provided    
    def rebuildAdsbDf(self):
        
        
        if 'ADSB' in self.dataType:
            adsbDate = self.dataDate
            self.adsbDateHyphen = '-'.join(adsbDate)
            self.adsbDate = ''.join(adsbDate)
            self.adsbDf = pd.read_csv('./ADSB/' + self.adsbDate +'_csv.txt')

    
#the refreshParameters method is an archaic method that can be used to change the objects altitude limits and time    
    def refreshParameters(self, adiLow, adiHigh, timeOut):
        
        self.adiLow = adiLow
        self.adiHigh = adiHigh
        self.timeOut = timeOut
        
#the refreshAltitude method is used to change the desired altitude limits
    def refreshAltitude(self, adiLow, adiHigh):
        
        self.adiLow = adiLow
        self.adiHigh = adiHigh

#the refreshTime method is used to change the desired time 
    def refreshTime(self, timeOut):
        self.timeOut = timeOut
                    
#the refreshDate method is used to change the desired date
    def refreshDate(self, date):
        self.dataDate = date
        
        
#the RGBtoHex method is used to convert RGB values[0:255, 0:255, 0:255] to workable hex values, this is needed in order to provide
#provide the ipyleaflet api with a color schema to represent flight paths and other visual objects
    def RGBtoHex(self, RGB):
        
        # Components need to be integers for hex to make sense
        RGB = [int(x) for x in RGB]
        return "#"+"".join(["0{0:x}".format(v) if v < 16 else
                "{0:x}".format(v) for v in RGB])


#the buildAdsbFlights method is used to build a list of all adsbFlights organized by their flight id
#it does not allow duplicates

    def buildAdsbFlights(self):
        

        flights = []
        flights.extend({self.adsbDf.iat[0,11]})

        for y in range(len(self.adsbDf)):

            acidCur = self.adsbDf.iat[y - 1 , 11]

            if(self.adsbDf.iat[y, 11] != acidCur ):

                if(acidCur in flights):
                    ;
                else:
                    flights.extend([acidCur])

        for i in flights:

            if pd.isnull(i) == True:
                flights.remove(i)

        self.adsbFlights = flights
        
        
#the refreshAdsbFlights method uses the adsbFlights list (containing the flight Id) to build
#a dictionary, organized into three seperate dictionaries within. These three seperate dictionaries
#are based on the flight Id, and are prefaced with that, but with the concatenated string of
#either 'track' , 'altitude', or 'time'. 

#adsbFlightsDictionary[flightId + 'track'] contains a list of the latitude and longitude coordinates
#adsbFlightsDictionary[flightId + 'time'] contains a list of the time stamps for those coordinates
#adsbFlightsDictionary[flightId + 'altitude'] contains a list of the altitudes of those coordinates
    
    def refreshAdsbFlights(self):

        adsbFlightsDictionary = {}
        adiLow = self.adiLow
        adiHigh = self.adiHigh
        timeOut = self.timeOut

        for items in self.adsbFlights:

            if type(items) == str:

                adsbFlightsDictionary.update({items+'track':[]})
                adsbFlightsDictionary.update({items+'time':[]})
                adsbFlightsDictionary.update({items+'altitude':[]})  
                
        for x in range(len(self.adsbDf)):

            timeIn = self.adsbDf.iat[x,1]
            timeIn = time.strftime('%H:%M:%S', time.gmtime(timeIn))
            timeIn = ' '.join([self.adsbDateHyphen, timeIn])
            
            if type(self.adsbDf.iat[x,11]) == str:
                
                
                if timeIn >= self.adsbDateHyphen + timeOut[0] and timeIn <= self.adsbDateHyphen + timeOut[1]:
                    if int(self.adsbDf.iat[x,7]) >= adiLow and int(self.adsbDf.iat[x,7]) <= adiHigh:
                        adsbFlightsDictionary[self.adsbDf.iat[x,11] + "track"].append([self.adsbDf.iat[x,4] , self.adsbDf.iat[x,5]])    
                        adsbFlightsDictionary[self.adsbDf.iat[x,11] + "time"].append(timeIn)
                        adsbFlightsDictionary[self.adsbDf.iat[x,11] + "altitude"].append([self.adsbDf.iat[x,10], self.adsbDf.iat[x,7]])
                        
        self.adsbFlightsDictionary = adsbFlightsDictionary

        
#the refreshAdsbDisplay method takes in flight tracks for each adsbFlight based on flightId
#it then creates a polyline, which is then assigned a color from the RGBtoHex method
##currently it assigns colors between [0,0,255] and [0,255,255] which is blue and teal
    
    def refreshAdsbDisplay(self):
        
        lp = 0
        lines = []
        colorRange = 255/len(self.adsbFlights)
        
        for items in self.adsbFlights:
            if lp == 0:
                colorIn = self.RGBtoHex([0,0,255])
            else:
                colorIn = self.RGBtoHex([ 0, int(255-(colorRange * lp)), 255 ])
            
            flightName = items
            
            items = Polyline(
            locations = 
                [[
                    self.adsbFlightsDictionary[items + 'track']
                ,]],
                color = colorIn,
                fill_color = colorIn,
                fill = False,
                opacity = .7)
            
            
            #this provides the polylines with clickable functionality, currently displaying their flightId on click
            if(type(self.adsbFlights[lp]) == str):
                items.popup=HTML(flightName)

            lines.append(items)
            lp+=1;
            
            
            
        self.adsbLines = lines
        if self.adsbDisplay == None:
            self.adsbDisplay = LayerGroup(layers=(lines)) 
        else:
            self.adsbDisplay.layers = self.adsbLines
            
            
            
#the refreshUasFlights method builds a dictionary named uasOd, it contains the flight tracks, timestamps, and altitudes for each flight based on 
#flight id for that aeroscope flight. additionally, it creates an entry for uasHeat, which is later used to determine horizontal distance
#delimiters for uasFlight waypoints


    def refreshUasFlights(self):
        flightTracks = []
        flightTimes = []
        flightAltitudes = []
        uasOd = {}

        timeMin = self.adsbDateHyphen + self.timeOut[0]

        
        timeMax = self.adsbDateHyphen + self.timeOut[1]
        
        
        adiLow = self.adiLow
        adiHigh = self.adiHigh

        
        for z in self.uasFlights:
            tracks = []
            times = []
            altitudes = []
            heatIterator = 0
            for i in range(len(self.uasFlights[z][0][0])):
                k = self.uasFlights[z][0][0][i]
                j = self.uasFlights[z][0][1][i]
                
                
                timeIn = self.uasFlights[z][1][i]
                
                
                
                altitude = self.uasFlights[z][2][i]
                if type(timeIn) == str:                    
                    if timeIn >= timeMin and timeIn <= timeMax:
                        timeIn = time.strftime(timeIn)
                        if altitude >= adiLow and altitude <= adiHigh:
                            tracks.append([k,j])
                            times.append(timeIn)
                            altitudes.append(altitude)
                            self.uasHeat.update({z + str(heatIterator):[]})
                            heatIterator+=1
    

            flightTimes.append(times)
            flightTracks.append(tracks)
            flightAltitudes.append(altitudes)
            #the below giving dictionary full of blanks
            uasOd[z] = (flightTimes, flightTracks,flightAltitudes)
            #uasNumberedDictionary[z + str(heatIterator)] = 
            self.uasOd = uasOd
            
            

#the refreshUasDisplay method, similar to the refreshAdsbDisplay uses the flightTracks for each aeroscope flight
#it then creates a polyline, which is then assigned a color from the RGBtoHex method
##currently it assigns colors between [255,0,0] and [255,0,255] which is red and pink/purple
    

    def refreshUasDisplay(self):
        
        
        lp = 0
        lines = []
        colorRange = 255/len(self.uasFlights)

        for i in self.uasFlights:
            if lp == 0:
                colorIn = self.RGBtoHex([255,0,0])
            else:
                colorIn = self.RGBtoHex([ 255 , 0, int(255-(colorRange * lp)) ])

            
            line = Polyline(
            locations = 
                [[
                    self.uasOd[i][1][lp]
                ,]],
                color = colorIn,
                fill_color = colorIn,
                fill = False,
                opacity = .7)

            line.popup=HTML(i)
            lines.append(line)


            lp+=1;
            
        self.uasLines = lines
        if self.uasDisplay == None:
            self.uasDisplay = LayerGroup(layers=(lines)) 
        else:
            self.uasDisplay.layers = self.uasLines
             
                
#the refreshUasHeatTree method takes an integer parameter heatDistanceDelimiter, it then builds a 
#quadtree using the pyqtree library. these entries into the quadtree are based on the coordinates
#of aeroscope waypoints

    def refreshUasHeatTree(self, heatDistanceDelimiter):
        
        
        self.heatDistanceDelimiter = heatDistanceDelimiter
        
        if self.heatDisplay == None:
            circles = []

            iterator = 0
            uasTree =  qt.Index(bbox= (-90, -180, 90, 180))
            for uasFlight in self.uasFlights:
                uasTreeCoordIndex = 0
                for coordinate in self.uasOd[uasFlight][1][iterator]:

                    
                    #conversion is used to change latitude and longitude to feet
                    latitudeFeetConversion = 6076 * 60
                    
                    xmin = ((coordinate[0] * 6076)*60 - self.heatDistanceDelimiter) / latitudeFeetConversion
                    xmax = ((coordinate[0] * 6076)*60 + self.heatDistanceDelimiter) / latitudeFeetConversion
                    #math.cos is used to correct the longitude calculation for the location of the latitude on the globe
                    ymin = ((coordinate[1] * 6076)*60 - (self.heatDistanceDelimiter * math.cos(math.radians(xmin)))) / latitudeFeetConversion
                    ymax = ((coordinate[1] * 6076)*60 + (self.heatDistanceDelimiter * math.cos(math.radians(xmax)))) / latitudeFeetConversion
                    self.uasHeat[uasFlight + str(uasTreeCoordIndex)] = (xmin, ymin, xmax, ymax)

                    uasTree.insert(uasFlight + str(uasTreeCoordIndex), self.uasHeat[uasFlight + str(uasTreeCoordIndex)])
                    uasTreeCoordIndex+=1

                    #a circle is built in the ipyleaflet library, this is used to display the distance around the waypoints to show the 
                    #corresponding distance of the heatDistanceDelimiter
                    #color of the circles is currently assigned 'darkred'
                    heatCircle = Circle()
                    heatCircle.location = coordinate
                    #heatDistance is divied by 3.281 to convert from meters to feet
                    heatCircle.radius = int(self.heatDistanceDelimiter / 3.281)
                    #the Circle.popup causes the circles to be clickable, and show the flightId along with the waypoint number of that flight 
                    heatCircle.popup = HTML(uasFlight + '\n waypt#' +str(uasTreeCoordIndex))
                    heatCircle.color = 'darkred'
                    heatCircle.opacity = .3
                    heatCircle.weight = 1
                    heatCircle.fill_color='darkred'
                    circles.append(heatCircle)

                iterator+=1

            self.heatCircles = circles
            self.uasTree = uasTree
            self.heatDisplay = LayerGroup(layers=(circles)) 
        else:
            iterator = 0
            uasTree =  qt.Index(bbox= (-90, -180, 90, 180))
            for uasFlight in self.uasFlights:
                uasTreeCoordIndex = 0
                for coordinate in self.uasOd[uasFlight][1][iterator]:

                    latitudeFeetConversion = 6076 * 60
                    
                    xmin = ((coordinate[0] * 6076)*60 - self.heatDistanceDelimiter) / latitudeFeetConversion
                    xmax = ((coordinate[0] * 6076)*60 + self.heatDistanceDelimiter) / latitudeFeetConversion
                    ymin = ((coordinate[1] * 6076)*60 - (self.heatDistanceDelimiter * math.cos(math.radians(xmin)))) / latitudeFeetConversion
                    ymax = ((coordinate[1] * 6076)*60 + (self.heatDistanceDelimiter * math.cos(math.radians(xmax)))) / latitudeFeetConversion
                    self.uasHeat[uasFlight + str(uasTreeCoordIndex)] = (xmin, ymin, xmax, ymax)

                    uasTree.insert(uasFlight + str(uasTreeCoordIndex), self.uasHeat[uasFlight + str(uasTreeCoordIndex)])
                    uasTreeCoordIndex+=1

                iterator+=1
            self.uasTree = uasTree  
            for heatCircle in self.heatCircles:
                heatCircle.radius = int(self.heatDistanceDelimiter / 3.281)
            self.heatDisplay.layers = self.heatCircles
        ;
        
        
#the refreshAdsbTree methods builds a quadtree of the adsbFlight coordinates for each respective flight    
        
    def refreshAdsbTree(self):
        

        adsbTree = qt.Index(bbox= (-90, -180, 90, 180))

        for flight in self.adsbFlights:
            flightTreeIndex = 0
            for coordinates in self.adsbFlightsDictionary[flight + 'track']:
                adsbTree.insert(flight + ',' + str(flightTreeIndex), (coordinates[0], coordinates[1], coordinates[0], coordinates[1]))
                flightTreeIndex+=1
        self.adsbTree = adsbTree
        
        
#the burnEncounterTrees method creates a list of potential encounters, which entails encounter only based on horizontal distance
#other methods need to be used in order to remove items from the list of potential encounters based on vertical distance and timestamps

    def burnEncounterTrees(self):
        
        encounterResults = []
        uasEncounters = []

        for uasHeat in self.uasHeat:

            result = self.adsbTree.intersect(self.uasHeat[uasHeat])
            if len(result) > 0:

                encounterResults.append(result)
                uasEncounters.append([result,uasHeat[:-1], uasHeat[-1]])
            
        self.encounterResults = encounterResults
        self.uasEncounters = uasEncounters
        
        encounterFlights = []
        uasEncounterFlights = []
        
        for uasEncounterList in self.encounterResults:
            for encounter in uasEncounterList:
                encounter = encounter.split(',')
                if encounter[0] in encounterFlights:
                    ;
                else:
                    encounterFlights.append(encounter[0])

        self.encounterFlights = encounterFlights
        ;

        
#the refreshEncounterDisplay method takes the list of potential encounters and changes the colors of the adsbFlights which intersect horizontally
#with the respective aeroscope waypoint. additionally, it stores the old color of that adsb flight path in a list called oldColors. it currently
#changes the color to gold if there is an intersection

    def refreshEncounterDisplay(self):
        oldColors = []
        colorShuffle = 0
        for name in self.adsbFlights:

            if self.adsbLines[self.adsbFlights.index(name)].color == 'gold':
                self.adsbLines[self.adsbFlights.index(name)].color = self.oldColors[colorShuffle][0]
                colorShuffle +=1

            if name in self.encounterFlights:
                oldColors.append((self.adsbLines[self.adsbFlights.index(name)].color, self.adsbFlights.index(name)))
                self.adsbLines[self.adsbFlights.index(name)].color = 'gold'
            else:
                ;
        self.oldColors = oldColors
        
#the altitudeCompare method is used to take the list of horizontal encounters and updates the list of encounters 
#based on vertical intersections by altitude.
#it currently is a work in progress, it works properly, but needs to be tested

    def altitudeCompare(self, altitudeDelimiter):
        
        self.altitudeDelimiter = altitudeDelimiter
        
        for i in list(self.uasEncounters[:]):

            altLow = min(self.uasFlights[i[1][:14]][2]) * 3.28084
            altHigh = max(self.uasFlights[i[1][:14]][2]) * 3.28084
            print(i[1][:14])
            print(i[1]+i[2])
            #print(i[0])
            print(altLow, altHigh)
            for z in i[:][0]:

                #pmax = max(self.uasFlights[i][2][:])
                #pmin = min(self.uasFlights[i][2][:])

                if (self.adsbFlightsDictionary[z.split(',')[0]+'altitude']
                [int(z.split(',')[1])]
                [1]) >altLow and (self.adsbFlightsDictionary[z.split(',')[0]+'altitude']
                [int(z.split(',')[1])]
                [1]) < altHigh:
                    print(self.adsbFlightsDictionary[z.split(',')[0]+'altitude']
                          [int(z.split(',')[1])])
                    print('pass')

                    #print(self.adsbFlightsDictionary[z.split(',')[0]+'altitude']
        #[int(z.split(',')[1])]
        #[1])
                    #print(altLow)
                    #print(altHigh)
                else:
                    print(self.adsbFlightsDictionary[z.split(',')[0]+'altitude']
                          [int(z.split(',')[1])])
                    i[0].remove(z)
                    print('removed', z , "from" , i[1][:14] + i[2])
                    print(' ')
                    #i[0].remove(z)

                    
#         for uasEncounterList in self.encounterResults:
#             for encounter in uasEncounterList:
#                 encounter = encounter.split(',')
#                 if encounter[0] in encounterFlights:
#                     ;
#                 else:
#                     encounterFlights.append(encounter[0])

#         self.encounterFlights = encounterFlights
            
        
        
        
#the timeCompare method is used to take the list of horizontal encounters and updates the list of encounters 
#based on time stamps of the flights.
#it currently is a work in progress, it does not function properly but needs to be tested. the timeDelimiter needs to be added
#as it was being tested with a default
           
    def timeCompare(self, timeDelimiter):
        
        self.timeDelimiter = timeDelimiter
        
        form = '%Y-%m-%d %H:%M:%S'

        #minnnn = min(self.uasFlights[i[1][:14]][2])
        #minnnn

        p = []


        for i in list(self.uasEncounters[:]):

            #altLow = min(self.uasFlights[i[1][:14]][2]) * 3.28084
            #lo = i[1]

            timeindex = int(i[1][14:] + i[2])
            #print(timeindex)


            timeLow = self.uasFlights[i[1][:14]][1][timeindex]
            print('uas')
            print(timeLow)

            for z in i[:][0]:
                print('adsb')
                print(self.adsbFlightsDictionary[z.split(',')[0]+'time']
                      [int(z.split(',')[1])])

                timeA = datetime.datetime.strptime(timeLow,form )
                timeB = datetime.datetime.strptime(self.adsbFlightsDictionary[z.split(',')[0]+'time']
                      [int(z.split(',')[1])], form)

                if timeA>timeB:
                    diff = timeA-timeB
                else:

                    diff = (timeB - timeA)
                print(diff)

                #the time default is currently 60 minutes, but the implementation needs to be tested
                if diff > timedelta(minutes = 60):
                    i[0].remove(z)
                    print('removed', z , "from" , i[1][:14] + i[2])




        
#the rebuildDf method clears the display of heat, and calls the corresponding methods to rebuild the display
#and dataframe based on newly entered parameters

    def rebuildDf(self):
        
        
        self.heatDisplay.clear_layers()
        self.heatDisplay = None        
        self.uasHeat = {}
        self.adsbTree = {}
        self.encounterList = None
        self.buildAdsbFlights()
        self.refreshAdsbFlights()
        self.refreshAdsbDisplay()
        self.refreshUasFlights()
        self.refreshUasDisplay()
        self.refreshAdsbTree()
        self.refreshUasHeatTree(0)
        self.burnEncounterTrees()
        self.refreshEncounterDisplay()
        
        if self.mapp == None:
            mapp.add_layer(self.heatDisplay)
        else:
            self.mapp.add_layer(self.heatDisplay)
            

        

In [ ]:
class mappConstructor():
    
#the mappConstructor constructor defaults the values specified below to pass to the mappDataFrame constructor
    def __init__(self):
        
        self.mdf = None
        self.airport = 'DAB'
        self.mapp = None
        self.zoom = 10
        self.dataType = ['ADSB', 'AERO', 'TFMS']
        self.dataDate = ['2019', '08', '24']
        self.altitudeLow = 0
        self.altitudeHigh = 4000
        self.time = [' 00:00:00', ' 23:59:59']
        self.encounterDisplay = None
        self.adsbSelector = None
        self.aeroSelector = None
        self.encounterList = None
        self.altitudeDisplay = None
        
        
#the buildDataFrame method calls 
    def buildDataFrame(self):
            
        
        mdf = mappDataFrame()
        mdf.buildDataFrames(self.dataType, self.dataDate, self.airport)
        mdf.refreshParameters(self.altitudeLow ,self.altitudeHigh, self.time)
        mdf.buildAdsbFlights()
        mdf.refreshAdsbFlights()
        mdf.refreshAdsbDisplay()
        mdf.refreshUasFlights()
        mdf.refreshUasDisplay()
        mdf.refreshAdsbTree()
        mdf.refreshUasHeatTree(0)
        mdf.burnEncounterTrees()
        mdf.refreshEncounterDisplay()
        
        self.mdf = mdf
        
    def construct(self):
        
        
        
        mapp = Map(
    layers=(basemap_to_tiles(basemaps.Stamen.Toner), ),
        zoom = self.zoom,
        )
        
        mapp.add_layer(self.mdf.adsbDisplay)
        mapp.add_layer(self.mdf.uasDisplay)
        mapp.add_layer(self.mdf.heatDisplay)
        mapp.add_control(FullScreenControl()) 
        
        if self.airport == 'DAB':
            mapp.center = [29.1832, -81.0532]
            
        elif self.airport == 'ABQ':
            mapp.center = [35.0433, -106.6129]
            
        self.mapp = mapp
        self.addWidgets()
        self.mdf.mapp = self.mapp
        
#the display method returns the mapp 

    def display(self):
        
        return self.mapp
    
#the rebuildMdf method takes the altitudeLow,altitudeHigh, timeLow, and timeHigh parameters and then
#calls the mappDataFrame.refreshParameters method along with the mappDataFrame.rebuildDf method

    def rebuildMdf(self, altitudeLow, altitudeHigh, timeLow, timeHigh):
    
        self.altitudeLow = altitudeLow
        self.altitudeHigh = altitudeHigh
        self.time = [timeLow, timeHigh]
        
        
        
        self.mdf.refreshParameters(self.altitudeLow, self.altitudeHigh, self.time)
        self.mdf.rebuildDf()
    
#the refreshEncounterDistance method takes the heatDistanceDelimiter and then passes it into the the mappDataFrame,
#it then calls the mappDataFrame.burnEncounterTrees method to intersect the adsbTree and the uasTree to 
#determine horizontal intersections, and then refreshes the display to change the colors of those adsb Flight Paths
    def refreshEncounterDistance(self, heatDistanceDelimiter):
    
        self.mdf.refreshUasHeatTree(heatDistanceDelimiter)
        self.mdf.burnEncounterTrees()
        self.mdf.refreshEncounterDisplay()

        
#the refreshEncounters method takes the altitudeDelimiter and passes it to the mappDataFrame and calls the 
#altitudeCompare method. 

    def refreshEncounters(self, altitudeDelimiter):
        
        self.mdf.altitudeCompare(altitudeDelimiter)
        
#the refreshEncountersTime method takes the timeDelimiter and passes it to the mappDataFrame and calls the 
#timeCompare method. This method works currently, but the timeCompare method is still under development    
            
    def refreshEncountersTime(self, timeDelimiter):
        
        self.mdf.timeCompare(timeDelimiter)
        
        
        
        
#the addWidgets method is a constructor that builds the widgets using the iPyWidgets API and adds them
#to the mapp in order to let the user manipulate them. 

    def addWidgets(self):
        
        
#encounterDisplay is the widget used to manipulate horizontal encounters
        
        encounterDisplay = widgets.BoundedIntText(
            value = 0,
            min = 0,
            max = 1000,
            step = 20,
            description = 'encounterDistance',
            disabled = False,
            continuous_update=True
        )

        encounterOutput = widgets.Output()

        def encounterChange(change):
            with encounterOutput:
                self.refreshEncounterDistance(encounterDisplay.value)
                self.encounterList.options = self.mdf.encounterFlights
                print(encounterDisplay.value)

        
        encounterDisplay.observe(encounterChange, names='value')
        self.encounterDisplay = encounterDisplay
        
        
#adsbSelector is the widget used to select which adsb flight paths to display on the mapp
        
        adsbSelector = widgets.SelectMultiple(
            options = self.mdf.adsbFlights,
            value = self.mdf.adsbFlights[0:2],
            description = 'adsbFlights',
            continuous_update = True)

        adsbFlightOutput = widgets.Output()

        def adsbChange(change):
            with adsbFlightOutput:
                self.mdf.adsbDisplay.clear_layers()
                for x in adsbSelector.options:
                    if ( str(x) in adsbSelector.value ):
                        self.mdf.adsbDisplay.add_layer(
                            self.mdf.adsbLines[self.mdf.adsbFlights.index(x)])     
                    else:
                        ;
        adsbSelector.observe(adsbChange, names = 'value')
        self.adsbSelector = adsbSelector

 #aeroSelector is the widget used to select which aeroscope flight paths to display on the map
    
        aeroSelector = widgets.SelectMultiple(
        options = list(self.mdf.uasFlights.keys()),
        value = list(self.mdf.uasFlights.keys())[0:2],
        description = 'djiaeroscope',
        continuous_update = True)

        aeroFlightOutput = widgets.Output()
        
        def aeroChange(change):
            with aeroFlightOutput:
                self.mdf.uasDisplay.clear_layers()
                for x in aeroSelector.options:
                    if ( str(x) in aeroSelector.value ):
                        self.mdf.uasDisplay.add_layer(
                            self.mdf.uasLines[list(self.mdf.uasFlights.keys()).index(x)])     
                    else:
                        ;
                
        
        aeroSelector.observe(aeroChange, names='value')
        self.aeroSelector = aeroSelector
        

#encounterList is the widget used to display the list of horizontal encounters
#these are only potential encounters currently, it can be used to display the list of verified 
#encounters once the altitudeCompare and timeCompare methods are properly implemented and tested

        encounterList = widgets.Dropdown(options = self.mdf.encounterFlights, 
                                         value=None, 
                                         continuous_update=True,
                                         description = 'encounterList')
        
        self.encounterList = encounterList
        

#altitudeDisplay is the widget used to call the refreshEncounters method, to 
#verify which horizontal encounters also intersect vertically. The widget functions 
#properly but the altitudeCompare method still needs testing
        
        altitudeDisplay = widgets.BoundedIntText(
            value = 0,
            min = 0,
            max = 1000,
            step = 10,
            description = 'altitudeDistance',
            disabled = False,
            continuous_update=False
        )
        
        altitudeOutput = widgets.Output()
        
        def altitudeChange(change):
            
            with altitudeOutput:
                
                self.refreshEncounters(altitudeDisplay.value)
                print(altitudeDisplay.value)
                
        altitudeDisplay.observe(altitudeChange, names = 'value')
        self.altitudeDisplay = altitudeDisplay

        
#timeDisplay is the widget which will pass the entered value to the refreshEncountersTime
#method. This widget needs testing, along with the implementation of the refreshEncountersTime
#method. Currently not functional

        timeDisplay = widgets.BoundedIntText(
            value = 0,
            min = 0,
            max = 1000,
            step = 10,
            description = 'timeDistance',
            disabled = False,
            continuous_update=False
        )
        
        timeOutput = widgets.Output()
        
        def timeChange(change):
            
            with timeOutput:
                
                self.refreshEncountersTime(timeDisplay.value)
                print(timeDisplay.value)
                
        timeDisplay.observe(timeChange, names = 'value')
        self.timeDisplay = timeDisplay
        
        accordion = widgets.Accordion(children=[adsbSelector,
                                                encounterDisplay,
                                                aeroSelector,
                                                altitudeDisplay,
                                               timeDisplay])
        accordion.set_title(0, 'adsb')
        accordion.set_title(1, 'encounter')
        accordion.set_title(2, 'aeroscope')
        accordion.set_title(3, 'altitude')
        accordion.set_title(4, 'time')
    
        
        widgetControl = WidgetControl(widget = accordion, position = 'bottomright')
        widgetControlLeft = WidgetControl(widget = encounterList, position = 'bottomleft')
        self.mapp.add_control(widgetControl)
        self.mapp.add_control(widgetControlLeft)
    

In [1015]:
#calls the mapp constructor method
mapp = mappConstructor()
#builds the mappDataFrame with the default values set within the class
mapp.buildDataFrame()

In [1016]:
#constructs the mapp
mapp.construct()

In [1017]:
#displays the mapp
mapp.display()

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

**FUTURE WORK**


1. Finalize altitudeCompare and timeCompare in order to take list of potential encounters and to return verified encounters

2. Implement GEOJson import to display the LAANC grid overtop of the airport

3. enable user to select which airport they would like to display, additionally, to enable timezone corrections for data based on the selected airport's timezone

4. implement a method which takes the verified encounters and changes the color of adsb flight paths to show encounters

5. utilize the matplotlib library in order to build a 3-d representation of individual encounters. This includes the aeroscope flight path along with the adsb flight path

6. create a method to count the number of flight paths within a specific square within the LAANC grid